In [6]:
import pandas as pd
import requests
from io import StringIO
import pymysql
import mysql.connector
from bs4  import BeautifulSoup
import numpy as np
import time

### Problem-1:

You are given a SQL file link: https://drive.google.com/file/d/1WFt7B84LTHhMueoKmz8W-PRo7xXqmZf3/view?usp=share_link. Read the data by using the file and store it in a excel file. In this data, there are 3 tables named "invoices", "order_leads" and "sales_sql". So create 3 sheets to your excel file.

In [3]:
# code here
con = mysql.connector.connect(host='localhost',
                              password='',
                              user='root',
                              database='world')
df1 = pd.read_sql_query('Select * from invoices', con=con)
df2 = pd.read_sql_query('Select * from order_leads', con=con)
df3 = pd.read_sql_query('Select * from sales_sql', con=con)

with pd.ExcelWriter('output.xlsx') as writer:
    df1.to_excel(writer,sheet_name='Sheet1')
    df2.to_excel(writer,sheet_name='Sheet2')    
    df3.to_excel(writer,sheet_name='Sheet3')    

C:\Users\Aniket\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\Aniket\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\Aniket\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


### Problem-2

Go to the site: https://rapidapi.com/wirefreethought/api/geodb-cities. From here, you have to grab the API and have to choose proper routes to get the cities of different countries. After getting the right API, hit that API and create a dataframe of all the cities that you can get by using the API. Then store the dataframe to a SQL. If you need to create an account or have to subscribe, then do that (it has free subscription but has some limitations. Use that free subscription and modify your code accordingly to get all the data).  

In [4]:
webpage = requests.get('https://en.wikipedia.org/wiki/List_of_ISO_3166_country_codes').text
soup = BeautifulSoup(webpage,'lxml')
cntry_id = soup.find_all('span',class_="monospaced")[::3]
l = []
for i in cntry_id:
    l.append(i.text)

In [7]:
### code here
df = pd.DataFrame()
capital = []
country = []
headers = {
        "X-RapidAPI-Key": "3836a34f5amsh46b314c38ab3c53p1117b4jsnc5d68e7da5d3",
        "X-RapidAPI-Host": "wft-geo-db.p.rapidapi.com" }

for i in l[:20]:

    url = f"https://wft-geo-db.p.rapidapi.com/v1/geo/countries/{i}"

    response = requests.get(url, headers=headers)

    response = response.json()
    try:
        capital.append(response['data']['capital'])
        country.append(response['data']['name'])
        
    except:
        capital.append(np.nan)
        country.append(np.nan)
        
    time.sleep(3)

In [8]:
df['country'] = country
df['capital'] = capital
df.dropna(inplace=True)
df

,country,capital
0,Afghanistan,Kabul
2,Albania,Tirana
3,Algeria,Algiers
5,Andorra,Andorra la Vella
6,Angola,Luanda
9,Antigua and Barbuda,Saint John's
10,Argentina,Buenos Aires
11,Armenia,Yerevan
13,Australia,Canberra
14,Austria,Vienna


In [9]:
import pymysql
from sqlalchemy import create_engine

In [12]:
engine = create_engine('mysql+pymysql://root:@localhost/world')
df.to_sql('country_cities',con=engine,if_exists='append')

15

### Problem 3:

Go to this url: https://www.flipkart.com/search?q=smartphones. This is the url to find phones in flipkart website. You have to extract the below things:
1. image url of the phone
2. name of the image
3. average ratings
4. total ratings
5. total reviews
6. discounted price
7. actual price

Extract all the phones which are available in this website. So you have to use the pagination concept. **Also after requesting every page through the url, please wait for a while (minimum 2-3 seconds), otherwise your IP address can be banned to access the flipkart website later.**

After collecting all the data, save that in a JSON file.

In [94]:
# code here
from bs4 import BeautifulSoup
import requests
import time

In [108]:
img = []
name = []
ratings = []
total_ratings = []
dis_price = []
reviews = []
actual_price = []
for i in range(1,3):
    webpage = requests.get('https://www.flipkart.com/search?q=smartphones&page={}'.format(i)).text
    soup = BeautifulSoup(webpage,'lxml')
    phones = soup.find_all('div',class_="_2kHMtA")
  
    for i in phones:
        img.append(i.find('img',loading="eager")['src'])
        name.append(i.find('div',class_="_4rR01T").text) 
        ratings.append(i.find('div',class_="_3LWZlK").text)
        total_ratings.append(i.find('span',class_="_2_R_DZ").text.split()[0])
        reviews.append(i.find('span',class_="_2_R_DZ").text.split()[3])
        dis_price.append(i.find('div',class_="_30jeq3 _1_WHN1").text)
        actual_price.append(i.find('div',class_="_3I9_wc _27UcVY").text)
    time.sleep(3)

In [112]:
phone = pd.DataFrame()
phone['name'] = name
phone['ratings'] = ratings
phone['total_ratings'] = total_ratings
phone['reviews'] = reviews
phone['dis_price'] = dis_price
phone['actual_price'] = actual_price
phone['image'] = img
phone

,name,ratings,total_ratings,reviews,dis_price,actual_price,image
0,"POCO C50 (Royal Blue, 32 GB)",4.2,"36,661","1,687","₹5,999","₹8,999",https://rukminim1.flixcart.com/image/312/312/x...
1,"POCO C50 (Country Green, 32 GB)",4.2,"36,661","1,687","₹5,999","₹8,999",https://rukminim1.flixcart.com/image/312/312/x...
2,"POCO M4 5G (Cool Blue, 64 GB)",4.2,"80,603","5,659","₹12,999","₹15,999",https://rukminim1.flixcart.com/image/312/312/l...
3,"POCO C51 (Royal Blue, 64 GB)",4.3,"18,916",770,"₹6,749","₹9,999",https://rukminim1.flixcart.com/image/312/312/x...
4,"POCO C55 (Forest Green, 64 GB)",4.2,"27,987","1,571","₹9,499","₹11,999",https://rukminim1.flixcart.com/image/312/312/x...
5,"POCO C55 (Power Black, 64 GB)",4.2,"27,987","1,571","₹9,499","₹11,999",https://rukminim1.flixcart.com/image/312/312/x...
6,"Infinix Smart 7 HD (Green Apple, 64 GB)",4.4,"9,010",444,"₹6,199","₹7,999",https://rukminim1.flixcart.com/image/312/312/x...
7,"POCO C51 (Power Black, 64 GB)",4.3,"18,916",770,"₹6,749","₹9,999",https://rukminim1.flixcart.com/image/312/312/x...
8,"REDMI A1+ (Light Blue, 32 GB)",4.1,"4,301",225,"₹6,599","₹10,999",https://rukminim1.flixcart.com/image/312/312/x...
9,"POCO M4 5G (Power Black, 64 GB)",4.2,"80,603","5,659","₹12,999","₹15,999",https://rukminim1.flixcart.com/image/312/312/l...
